In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

from __future__ import division
import sys
import os
from os.path import join as pjoin

import numpy as np
from sklearn.model_selection import train_test_split
from glob import glob
sys.path.append('..')

In [2]:
from utils.commands import unzip, make_data_dir, mkdir, call, count_file, KaggleCLI
cli = KaggleCLI('cifar-10')

/usr/local/miniconda2/envs/kaggle/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
data_path = 'data'
model_path = 'models'
sample_path = pjoin(data_path, 'sample')

## Data preprocess

In [8]:
cli.download_data()

In [7]:
print os.getcwd()

/Users/dorian/WorkSpace/kaggle_fun/cifar_10


In [9]:
unzip(pjoin(data_path, 'test.7z'), data_path)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/miniconda2/envs/kaggle/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/miniconda2/envs/kaggle/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/usr/local/miniconda2/envs/kaggle/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/local/miniconda2/envs/kaggle/lib/python2.7/inspect.py", line 1051, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/local/miniconda2/envs/kaggle/lib/python2.7/inspect.py", line 1011, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/local/miniconda2/envs/kaggle/lib/python2.7/inspect.py", line 453, in getsourcefile

IndexError: string index out of range

In [5]:
unzip(pjoin(data_path, 'test.7z'), data_path)
unzip(pjoin(data_path, 'train.7z'), data_path)

0

In [28]:
for base_path in [data_path, sample_path]:
    for category in ['dogs', 'cats']:
        for folder in ['train', 'valid']:
            mkdir(os.path.join(base_path, folder, category))
    mkdir(pjoin(base_path, 'test', 'unknown'))

In [12]:
cwd = os.getcwd()
os.chdir('data/train/')

call("find . -name 'cat.*' | xargs -J ^ mv ^ cats")
call("find . -name 'dog.*' | xargs -J ^ mv ^ dogs")
os.chdir(cwd)

os.chdir('data/test/')
call("find . -name '*.jpg' | xargs -J ^ mv ^ unknown")

os.chdir(cwd)

In [13]:
cwd = os.getcwd()
os.chdir('data/')

train_cats, valid_cats, train_dogs, valid_dogs = train_test_split(os.listdir('train/cats'), os.listdir('train/dogs'), test_size=0.2)
# train_cats, test_cats, train_dogs, test_dogs = train_test_split(train_cats, train_dogs, test_size=0.1)

# training data
for d in valid_dogs:
    call("mv train/dogs/{} valid/dogs".format(d))
for c in valid_cats:
    call("mv train/cats/{} valid/cats".format(c))

# for d in test_dogs:
#     call("mv train/dogs/{} test/dogs".format(d))
# for c in test_cats:
#     call("mv train/cats/{} test/cats".format(c))


# sample data
for d in train_dogs[:20]:
    call("cp train/dogs/{} sample/train/dogs".format(d))
for c in train_cats[:20]:
    call("cp train/cats/{} sample/train/cats".format(c))

for d in valid_dogs[:5]:
    call("cp valid/dogs/{} sample/valid/dogs".format(d))
for c in valid_cats[:5]:
    call("cp valid/cats/{} sample/valid/cats".format(c))

from random import sample
for d in sample(os.listdir('test/unknown'), 10):
    call("cp test/unknown/{} sample/test/unknown/".format(d))

os.chdir(cwd)

## Fine tune VGG

In [4]:
data_path = sample_path

In [5]:
from utils.pretrained_models import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [6]:
vgg_model = VGG16.get_model(2).model

In [7]:
train_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
train_flow = train_datagen.flow_from_directory(
        os.path.join(data_path, 'train'),
        target_size=(224, 224),
        batch_size=5,
        class_mode='categorical')
valid_flow = valid_datagen.flow_from_directory(
        os.path.join(data_path, 'valid'),
        target_size=(224, 224),
        batch_size=5,
        class_mode='categorical')
test_flow = test_datagen.flow_from_directory(
        os.path.join(data_path, 'test'),
        target_size=(224, 224),
        batch_size=5,
        class_mode='categorical',
        shuffle=False)

Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Found 10 images belonging to 1 classes.


In [8]:
for l in vgg_model.layers[:-1]:
    l.trainable = False

In [11]:
vgg_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
checkpointer = ModelCheckpoint(pjoin(model_path, 'weights_best.hdf5'), save_best_only=True)
vgg_model.fit_generator(
        train_flow,
        steps_per_epoch=10,
        epochs=1,
        validation_data=valid_flow,
        validation_steps=10,
        callbacks=[checkpointer])

Epoch 1/1
10/10 [==============================] - 86s 9s/step - loss: 0.0054 - acc: 1.0000 - val_loss: 3.3346e-05 - val_acc: 1.0000


In [13]:
vgg_model.load_weights(pjoin(model_path, 'weights_best.hdf5'))
vgg_model.evaluate_generator(valid_flow)

[3.3346356303809443e-05, 1.0]

In [14]:
preds = vgg_model.predict_generator(test_flow)

In [18]:
isdog = preds[:,1]
ids = np.array([int(f.split('.')[-2].split('/')[1]) for f in test_flow.filenames])
submission = np.stack([ids, isdog], axis=1)

In [19]:
result_path = pjoin(data_path, 'submission.csv')
np.savetxt(result_path, submission, fmt='%d, %.5f', header='id,label', comments="")

In [ ]:
cli.submit_result(result_path)